# Introduction to Opik Agent Optimizer

You will need:

1. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
2. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [1]:
%pip install opik-optimizer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.1/126.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.5/681.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

Let's make sure we have the correct version:

In [2]:
import opik_optimizer

opik_optimizer.__version__

'1.0.6'

The version should be 1.0.6 or greater.

[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

Enter your Comet API key, followed by "Y" to use your own workspace:

In [3]:
import opik

# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "doug-comet" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [4]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


To optimize any prompt, we'll need three basic things:

1. A starting prompt
2. A metric
3. A dataset

## The Dataset

In this experiment, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

**NOTE:** The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [19]:
from opik_optimizer.demo import get_or_create_dataset

dataset = get_or_create_dataset("hotpot-300")

/tmp/ipython-input-70379200.py:3: DeprecationWarning: This function is deprecated. Please use the datasets directly from opik_optimizer.datasets module instead. For example: opik_optimizer.datasets.truthful_qa() or opik_optimizer.datasets.rag_hallucination()
  dataset = get_or_create_dataset("hotpot-300")


Let's take a look at some dataset items:

In [20]:
rows = dataset.get_items()
rows[0]

{'id': '0198ccfe-efba-7d28-acad-727ea577d76c',
 'question': 'Are Smyrnium and Nymania both types of plant?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [7]:
rows[1]

{'id': '0198ccfe-efb9-70ff-a616-61889599f2e4',
 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?',
 'answer': 'Walt Disney Productions'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [8]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [9]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the `output` of the LLM and the `reference` (the truth):

In [10]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0198cd48-3025-7dd2-a0af-2ac01850a7b3&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [11]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [12]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))

0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necesary configurations for using an Opik Agent Optimizer, you'll need three things:

1. An initial prompt
2. A metric wrapper

We're going to start with a pretty bad prompt... so we can optimize it!

In [13]:
from opik_optimizer import ChatPrompt

initial_prompt = ChatPrompt(
    system="Provide an answer to the question",
    user="{question}",
)

The metric wrapper:

In [15]:
def levenshtein_ratio(dataset_item, llm_output):
    metric = LevenshteinRatio(project_name=project_name)
    return metric.score(
        reference=dataset_item["answer"], # This must match dataset field
        output=llm_output,
)

As you can see the metric wrapper is composed of our chosen metric. It takes a dataset item (a dictionary row from the dataset) and the output from the LLM. You should replace "answer" when using a different dataset.

## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [16]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    model="openai/gpt-4o",  # LiteLLM name
    min_examples=3,
    max_examples=8,
    n_threads=4,
    seed=42,
)

Ok, let's optimize that prompt!

In [21]:
result = optimizer.optimize_prompt(
    prompt=initial_prompt,
    dataset=dataset,
    metric=levenshtein_ratio,
    n_trials=10,
    n_samples=50,
)

╭────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - FewShotBayesianOptimizer               │
│                                                                    │
│ -> View optimization details ]8;id=855572;https://www.comet.com/opik/api/v1/session/redirect/optimizations/?optimization_id=0198cd53-9f2d-7650-910e-d0bc21b526dd&dataset_id=0198ccfe-ec43-7649-97ad-fe00f0d2d788&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\                │
╰────────────────────────────────────────────────────────────────────╯


> Let's optimize the prompt:

╭─ system ───────────────────────────────────────────────────────────╮
│                                                                    │
│  Provide an answer to the question                                 │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭─ user ───────────

Output()

  Baseline score was: 0.1529.

> Let's add a placeholder for few-shot examples in the messages:
│    Created the prompt template:
│
│    ╭─ system ───────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  Provide an answer to the question.                                │
│    │                                                                    │
│    │  ## Examples                                                       │
│    │                                                                    │
│    │  FEW_SHOT_EXAMPLE_PLACEHOLDER                                      │
│    │                                                                    │
│    ╰────────────────────────────────────────────────────────────────────╯
│    ╭─ user ─────────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  {question}              

Output()

│    Trial 1 - score was: 0.3231 (111.27%).
│
│ - Starting optimization round 2 of 10
│
│    ╭─ system ───────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  Provide an answer to the question.                                │
│    │                                                                    │
│    │  ## Examples                                                       │
│    │                                                                    │
│    │  ### Example                                                       │
│    │                                                                    │
│    │  Question: Ramfis Trujillo was a close friend of the racecar       │
│    │  driver and diplomat of what nationality?                          │
│    │  Answer: Dominican                                                 │
│    │                                                                    │


...

│    Trial 8 - score was: 0.3991 (160.97%).
│
│ - Starting optimization round 9 of 10
│
│    ╭─ system ───────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  Provide an answer to the question.                                │
│    │                                                                    │
│    │  ## Examples                                                       │
│    │                                                                    │
│    │  ### Example                                                       │
│    │                                                                    │
│    │  Question: What American singer born 1951 does Jim Rooney  have    │
│    │  credits for?                                                      │
│    │  Answer: Iris Luella DeMent                                        │
│    │                                                                    │


Output()

│    Trial 9 - score was: 0.2753 (80.06%).
│
│ - Starting optimization round 10 of 10
│
│    ╭─ system ───────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  Provide an answer to the question.                                │
│    │                                                                    │
│    │  ## Examples                                                       │
│    │                                                                    │
│    │  ### Example                                                       │
│    │                                                                    │
│    │  Question: Who is the wife of Charlemagne who is a step mother to  │
│    │  Pepin the Hunchback?                                              │
│    │  Answer: Hildegard                                                 │
│    │                                                                    │


Output()

│    Trial 10 - score was: 0.3185 (108.27%).
│

> Optimization complete

╭─ Optimization results ─────────────────────────────────────────────╮
│                                                                    │
│  Prompt was optimized and improved from 0.1529 to 0.4317           │
│  (182.34%)                                                         │
│                                                                    │
│  Optimized prompt:                                                 │
│  ╭─ system ─────────────────────────────────────────────────────╮  │
│  │                                                              │  │
│  │  Provide an answer to the question.                          │  │
│  │                                                              │  │
│  │  ## Examples                                                 │  │
│  │                                                              │  │
│  │  ### Example                                                 │  │
│  │

In [22]:
result.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:             FewShotBayesianOptimizer                                                                 ║
║ Model Used:            openai/gpt-4o                                                                            ║
║ Metric Evaluated:      levenshtein_ratio                                                                        ║
║ Initial Score:         0.1529                                                                                   ║
║ Final Best Score:      0.4317                                                                                   ║
║ Total Improvement:     182.34%                                                                                  ║
║ Rounds Completed:      0                                              

In [24]:
result.get_run_link()

'https://www.comet.com/opik/api/v1/session/redirect/optimizations/?optimization_id=0198cd53-9f2d-7650-910e-d0bc21b526dd&dataset_id=0198ccfe-ec43-7649-97ad-fe00f0d2d788&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw=='

Well done optimizer!

You should see the percentage correct going from about 15% to about 50% (or more!) correct.

What did we find? The result is a series of messages:

In [25]:
result.details["chat_messages"]

[{'role': 'system',
  'content': "Provide an answer to the question.\n\n## Examples\n\n### Example\n\nQuestion: One Night in Istanbul stars a British actor and recording artist who is best known for his portrayal of Jonah Kirby on what BBC One drama?\nAnswer: Waterloo Road\n\n\n### Example\n\nQuestion: Alicia Gräfin is best known for her role in a war film directed by who?\nAnswer: David Ayer\n\n\n### Example\n\nQuestion: What comedy film from 1994 starred actress Heather Graham alongside James Le Gros?\nAnswer: Don't Do It\n\n\n### Example\n\nQuestion: Are Hüsker Dü and The Colourist both musical bands?\nAnswer: yes\n\n\n### Example\n\nQuestion: Who was the son of British radio and television presenter and historian best known as an analyst of election results?\nAnswer: Dan Snow\n\n\n### Example\n\nQuestion: Rectrix Aviation's jet charter business involves what service?\nAnswer: renting an entire aircraft\n\n\n### Example\n\nQuestion: What U.S. court presided over both the Slaughter-H

## Opik Visualization UI

When you create an Optimization Run, you'll see it in the Opik UI (either running locally or hosted).

If you go to your `comet.com/opik/YOURID/optimizations` page, you'll see your run at the top of the list:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-ui.png" width="500px">

Along the top of the page you'll see a running history of the metric scores, with the latest dataset selected.

If you click on your run, you'll see the set of trials that ran durring this optimization run. Running across the top of this page are the scores just for this trial. On the top right you'll see the best so far:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimize-trials.png" width="500px">

If you click on a trial, you'll see the prompt for that trial:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-prompt.png" width="500px">

From the trial page you can also see the Trial items, and even dig down (mouse over the "Evaluation task" column on a row) to see the traces.

## Using Optimized Prompts

How can we use the optimized results?


Once we have the "chat_messages", we can do the following:

In [26]:
from litellm.integrations.opik.opik import OpikLogger
import litellm

opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]


def query(question, chat_messages):
    messages = chat_messages[:-1]  # Cut off the last one
    # replace it with question in proper format:
    messages.append({"role": "user", "content": '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [27]:
query("When was David Chalmers born?", result.details["chat_messages"])

'Answer: April 20, 1966'

In [28]:
query("What weighs more: a baby elephant or an SUV?", result.details["chat_messages"])

'Answer: a baby elephant'

If it says "elephant" that is not correct!

We'll need to use an agent with tools to get a better answer.

# Next Steps

You can try out other optimizers. More details can be found in the [Opik Agent Optimizer documentation](https://www.comet.com/docs/opik/agent_optimization/overview).